In [2]:
from sklearn.model_selection import train_test_split
from ipynb.fs.full.EEG_Toolbox import *
from sklearn.preprocessing import StandardScaler , LabelEncoder
import sys, io
import pandas as pd 
import pickle
from ipynb.fs.full.Data_Processing import *
from ipynb.fs.full.evaluation import *
from braindecode.datasets.xy import create_from_X_y
import time
import numpy as np
import torch
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet , Deep4Net
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
from collections import namedtuple
import pickle

In [7]:
window_size_list = [1,2,5,10,15,30]
results = []
label = 'attention'
window_size_samples = 30

saved_file = "/cs/home/ybk1/Dissertation/data/all_users_sampled_30_window_annotated_EEG.pickle"
with open(saved_file, 'rb') as handle:
    all_tests = pickle.load(handle)
users = all_tests.keys()

for user in users:
    print("Working on user {0}".format(user))
    torch.backends.cudnn.benchmark = True
    dt = all_tests[user]

    X = np.array(dt["inputs"]).transpose(0,2,1).astype(np.float32)
    y = dt[label]

    #Convert the categories into labels 
    le = LabelEncoder()
    y =  le.fit_transform(y)

    print(X.shape)
    print(y.shape)


    # Implement Train Test Split
    time_start = time.time()
    train_X, X_test, train_y, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
    train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.20, random_state=42, stratify=train_y)

    # standardize per channel
    means = train_X.mean(axis=(0,2), keepdims=True)
    stds = train_X.std(axis=(0,2), keepdims=True)
    train_X = (train_X - means) / (stds)
    valid_X = (valid_X - means) / (stds)
    X_test = (X_test - means) / (stds)

    save_stdout = sys.stdout
    sys.stdout = open('/cs/tmp/ybk1/trash', 'w')
    train_set = create_from_X_y(train_X, train_y, drop_last_window=False)
    valid_set = create_from_X_y(valid_X, valid_y, drop_last_window=False)
    sys.stdout = save_stdout

    print('Datasets created! Time elapsed: {} seconds'.format(time.time()-time_start))

    cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
    device = 'cuda' if cuda else 'cpu'
    if cuda:
        torch.backends.cudnn.benchmark = True
    seed = 20200220  # random seed to make results reproducible
    # Set random seed to be able to reproduce results
    set_random_seeds(seed=seed, cuda=cuda)

    n_classes= len(le.classes_)

    # Extract number of chans and time steps from dataset
    n_chans = train_set[0][0].shape[0]
    input_window_samples = train_set[0][0].shape[1]

    # model =  ShallowFBCSPNet(
    #     n_chans,
    #     n_classes,
    #     input_window_samples=input_window_samples,
    #     final_conv_length="auto",
    # )

    """
    For 30 samples, filter time_length = 1
    For 60 > samples, filter time length is left empty
    for 15 samples, filter_time length = 1, filter_length_2 = 1, filter_length_3 = 1
    """
    model =  Deep4Net(
        n_chans,
        n_classes,
        input_window_samples=input_window_samples,
        final_conv_length="auto", pool_time_length=1, filter_time_length = 1,
    #     filter_length_2  = 1, 
    #     filter_length_3 = 1,
            pool_time_stride=1,
    )

    # Send model to GPU
    if cuda:
        model.cuda()

    time_start = time.time()

    # These values we found good for shallow network:
    # lr = 0.0625 * 0.01
    # weight_decay = 0

    # For deep4 they should be:
    lr = 1 * 0.01
    weight_decay = 0.5 * 0.001

    batch_size = 32
    n_epochs = 50

    sys.stdout = open('/cs/tmp/ybk1/trash', 'w')

    clf = EEGClassifier(
        model,
        criterion=torch.nn.NLLLoss,
        optimizer=torch.optim.AdamW,
        train_split=predefined_split(valid_set),  # using valid_set for validation
        optimizer__lr=lr,
        optimizer__weight_decay=weight_decay,
        batch_size=batch_size,
        callbacks=[
            "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
        ],
        device=device,
    )

    # Model training for a specified number of epochs. `y` is None as it is already supplied
    # in the dataset.
    clf.fit(train_set, y=None, epochs=n_epochs)
    sys.stdout = save_stdout
    print('Training completed created! Time elapsed: {} seconds'.format(time.time()-time_start))

    y_pred = le.inverse_transform(clf.predict(valid_X))
    y_true = le.inverse_transform(valid_y)
    r = get_results(y_true, y_pred)


    # get results
    Results = namedtuple("Results","user label n_epochs window_size time num_classes size accuracy bal_acc precision recall f1_score_macro f1_score_micro")
    results.append(Results(user, label, n_epochs, window_size_samples, time.time()-time_start,  n_classes,len(X), r['Accuracy'], r['Balanced Accuracy'], r['Precision'], r['Recall'], 
                          r['F1 Score Macro'], r['F1 Score Micro']))
    
    # plot confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, le.classes_, normalize=True)
    plt.savefig("results/CNN/confusion/User_{0}_{1}.png".format(user,label))
    #plot loss curve
    plot_loss_curve(clf)
    plt.savefig("results/CNN/loss curves/User_{0}_{1}.png".format(user,label))
    print("Finished analysis on User {0}".format(user))
results  = pd.DataFrame(results).to_csv("results/CNN/tabulated/DeepCNN_Valid_performance_window_size_{0}_{1}_withclasses.csv".format(window_size_samples,label), index=False )

Working on user 1
(2832, 8, 30)
(2832,)
Datasets created! Time elapsed: 9.665364980697632 seconds
Loading data for 1 events and 30 original time points ...
Loading data for 1 events and 30 original time points ...


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [3]:
# # save model 
# saved_file = "models/attention_user_1_with_fi.pickle"
# with open(saved_file, 'wb') as handle:            
#     pickle.dump(clf, handle, protocol=pickle.HIGHEST_PROTOCOL)

<function RandomState.f>